In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

import tensorflow as tf
from keras import layers, Sequential, optimizers

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

for col in ['sex', 'smoker', 'region']:
    print(dataset[col].unique())
df = dataset.copy()


# encode binary feature
df['sex'].replace({'male': 1, 'female': 0}, inplace=True)
df['smoker'].replace({'yes': 1, 'no': 0}, inplace=True)
df

In [ ]:

fig, axes = plt.subplots(1, 4, figsize=(20, 4))

sns.boxplot(x='age', data=df, ax=axes[0])
sns.boxplot(x='bmi', data=df, ax=axes[1])
sns.boxplot(x='children', data=df, ax=axes[2])
sns.boxplot(x='expenses', data=df, ax=axes[3])

plt.show()

In [ ]:
train_dataset = df.sample(frac=0.8, random_state=0)
test_dataset = df.drop(train_dataset.index)

train_labels = train_dataset.pop('expenses')
test_labels = test_dataset.pop('expenses')


BUILD A MODEL

In [ ]:
binary_feature = ['sex', 'smoker']
categorical_feature = ['region']


inputs = {}
for name, column in train_dataset.items():
    if type(column[0]) == str:
        dtype = tf.string
    elif (name in categorical_feature or
          name in binary_feature):
        dtype = tf.int64
    else:
        dtype = tf.float32
    inputs[name] = tf.keras.Input(shape=(), name=name, dtype=dtype)


inputs



In [ ]:
preprocessed = []

for name in binary_feature:
    inp = inputs[name]
    inp = inp[:, tf.newaxis]
    float_value = tf.cast(inp, tf.float32)
    preprocessed.append(float_value)

preprocessed


In [ ]:
def stack_dict(inputs, fun=tf.stack):
    values = []
    for key in sorted(inputs.keys()):
        values.append(tf.cast(inputs[key], tf.float32))
    return fun(values, axis=-1)


numeric_feature_names = ['age', 'bmi', 'children']
numeric_features = train_dataset[numeric_feature_names]
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(stack_dict(dict(numeric_features)))


# Stacks the numeric features and runs them through the normalization layer
numeric_inputs = {}
for name in numeric_feature_names:
    numeric_inputs[name]=inputs[name]

numeric_inputs = stack_dict(numeric_inputs)
numeric_normalized = normalizer(numeric_inputs)
preprocessed.append(numeric_normalized)

preprocessed

In [ ]:
name = categorical_feature[0]
vocab = sorted(set(train_dataset[name]))
print(f'name: {name}')
print(f'vocab: {vocab}\n')

lookup = tf.keras.layers.StringLookup(vocabulary=vocab, output_mode='one_hot')
x = inputs[name][:, tf.newaxis]
x = lookup(x)
preprocessed.append(x)

In [ ]:

# Assemble the preprocessing head
preprocesssed_result = tf.concat(preprocessed, axis=-1)
preprocesssed_result
preprocessor = tf.keras.Model(inputs, preprocesssed_result)
preprocessor(dict(train_dataset.iloc[:1]))

body = Sequential([
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1)
])



x = preprocessor(inputs)
output = body(x)
model = tf.keras.Model(inputs, output)



In [ ]:

model.compile(
    optimizer=optimizers.Adam(learning_rate=0.1),
    loss='mean_absolute_error'
)

history = model.fit(
    dict(train_dataset),
    train_labels,
    epochs=30,
    verbose=1,
    validation_split = 0.2
)

In [ ]:

def plot_loss(history):
    plt.plot(history.history['loss'], label='loss', color='blue')
    plt.plot(history.history['val_loss'], label='val_loss', color='red')
    plt.xlabel('Epoch')
    plt.ylabel('Error')
    plt.ylim(bottom=0)
    plt.locator_params(integer=True)
    plt.legend()

plt.figure(figsize=(6, 6))
plot_loss(history)

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
mae = model.evaluate(dict(test_dataset), test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(dict(test_dataset)).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
